In [1]:
import pandas as pd

In [15]:
metadata = pd.read_csv("../data/03_ICIM_analysis/KC_ICIM_TSNE_data.csv")

In [16]:
metadata['dna_index'] = metadata.CellID.str.split('-').str[0]

In [17]:
metadata.to_csv("../data/06_tss_data/umi_with_metadata.csv")

In [18]:
metadata = metadata.drop(['nGene', 'nUMI', 'cell_type_id', 'is_kc', 'x', 'y', 'CellID'], axis=1)
metadata = metadata.set_index('dna_index')

In [25]:
cell_number_info = pd.read_csv("../data/06_tss_data/indexed_barcodes.tsv", sep='\t', header=None)

In [36]:
cell_number_info.columns = ['cell_number', 'dna_index']

In [40]:
cell_number_info = cell_number_info.set_index("dna_index")

In [42]:
metadata = metadata.join(cell_number_info)

In [44]:
metadata.to_csv("../data/06_tss_data/umi_with_metadata.csv")